In [26]:
import json
from tqdm import tqdm
import itertools
import pdb
import glob
import os
from PIL import Image

In [27]:
def divid_train_val_image(image_dir, img_ext, val_ratio):
    files = glob.glob(os.path.join(image_dir, "*" + img_ext))
    val_files = []
    train_files = []
    val_ratio_rpl = 1/val_ratio
    sum_ration = 0
    for idx, file in enumerate(files):
        sum_ration += val_ratio
        if sum_ration > 1:
            sum_ration = 0
            val_files.append(file)
        else:
            train_files.append(file)
            
    return train_files, val_files
    
    
image_dir = r"L:\Datasets\ocean_city\processed_json"
img_ext = ".json"
val_ratio = 0.2

train_files, val_files = divid_train_val_image(image_dir, img_ext, val_ratio)
print(len(train_files), len(val_files))

138 27


In [31]:
def create_image_json(files):
#     with open('gqa_image_metadata.json', 'w') as outfile:  
#     json.dump(gqa_image_data, outfile)
    results = []
    for file in files:
        file = file.replace(".json", ".jpg")
        img = Image.open(file)
        
        img_name = os.path.basename(file).replace(".jpg", "")
        img_h, img_w = img.size
        # print(img_w, img_h)
        data_dict = {"image_id": img_name,
                     "width": img_w,
                     "height": img_h,
                    "url": None, 
                    "flickr_id": None,
                    "coco_id": None,
                    "anti_prop": 0.0}
        results.append(data_dict)
        
    print(results)


# for image_set in (train_files, val_files):
create_image_json(train_files)

[{'image_id': '100_oUUwCL6RJ53TmRjrP5u1SQ_32.55', 'width': 3328, 'height': 3328, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '101_ZfvvGESbNnVG8yFN9PORqQ_24.2', 'width': 3328, 'height': 3328, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '102_aDNd7B-rqtKMpcw-Nvp5JQ_225.11', 'width': 4004, 'height': 4004, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '103_cC3D4p-Lj3pUVQ03gdovQw_45.33', 'width': 4096, 'height': 4096, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '104_izMCA8QLT_KCRokDsm_Zvg_319.71', 'width': 3328, 'height': 3328, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '106_LdXGcrXpLQc28rFjF__cKQ_0.43', 'width': 3328, 'height': 3328, 'url': None, 'flickr_id': None, 'coco_id': None, 'anti_prop': 0.0}, {'image_id': '107_7HfRX3diZje3jJgELHi5Kg_238.41', 'width': 3328, 'height': 3328, 'url': None, 'flickr_id': None

In [12]:
def create_gqa_image_data(gqa_image_data, scene_graphs, forb_list):

    for img in scene_graphs.keys():
        if img not in forb_list:
            gqa_image_data.append({'width': scene_graphs[img]['width'],
                                   'url' : None,
                                   'height': scene_graphs[img]['height'],
                                   'image_id': int(img),
                                   'coco_id' : None,
                                   'flickr_id' : None,
                                   'anti_prop' : 0.0})
    return gqa_image_data

In [3]:
gqa_image_data = []

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)    
    
    gqa_image_data = create_gqa_image_data(gqa_image_data, obj_data, [])

with open('gqa_image_metadata.json', 'w') as outfile:  
    json.dump(gqa_image_data, outfile)
    

In [4]:
def create_obj_dict(img_list, obj_data, forb_list):
    
    for img in obj_data.keys():
        if img not in forb_list:
            tmp_dict = {'image_id' : int(img), 'objects' : [], 'image_url' : ''}
            tmp_list = []
            for obj in obj_data[img]['objects']:
                tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                 'h': obj_data[img]['objects'][obj]['h'],
                                 'object_id': int(obj),
                                 'merged_object_ids': [],
                                 'names': [obj_data[img]['objects'][obj]['name']],
                                 'w': obj_data[img]['objects'][obj]['w'],
                                 'y': obj_data[img]['objects'][obj]['y'],
                                 'x': obj_data[img]['objects'][obj]['x']})

            tmp_dict['objects'] = tmp_list
            img_list.append(tmp_dict)
        
    return img_list

In [5]:
def create_rel_dict(cntr, rel_list, obj_data, forb_list):

    for img in obj_data.keys():
        if img not in forb_list:
            tmp_list = []
            for obj in obj_data[img]['objects']:
                for rel in obj_data[img]['objects'][obj]['relations']:
                    cntr += 1
                    subj = rel['object']
                    tmp_list.append({'predicate': rel['name'],
                                     'object': {'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'merged_object_ids': [],
                                     'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x'],
                                     'names': [obj_data[img]['objects'][obj]['name']]},
                                    'relationship_id': cntr,
                                    'synsets': [rel['name']],
                                    'subject': {'name': obj_data[img]['objects'][subj]['name'],
                                     'h': obj_data[img]['objects'][subj]['h'],
                                     'synsets': [obj_data[img]['objects'][subj]['name']],
                                     'object_id': int(subj),
                                     'w': obj_data[img]['objects'][subj]['w'],
                                     'y': obj_data[img]['objects'][subj]['y'],
                                     'x': obj_data[img]['objects'][subj]['x']}})
            rel_list.append({'relationships' : tmp_list, 'image_id' : int(img)})
    
    return cntr, rel_list

In [6]:
def create_attr_dict(attr_list, obj_data, forb_list):
    
    for img in obj_data.keys():
        if img not in forb_list:
            tmp_dict = {'image_id' : int(img), 'attributes' : []}
            tmp_list = []
            for obj in obj_data[img]['objects']:
                if obj_data[img]['objects'][obj]['attributes'] != []:
                    tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'attributes' : obj_data[img]['objects'][obj]['attributes'],
                                     'names': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x']})
                else:
                    tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'names': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x']})

            tmp_dict['attributes'] = tmp_list
            attr_list.append(tmp_dict)
        
    return attr_list

In [7]:
def save_to_file(text, name):

    with open(name, mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(text))
        myfile.write('\n')

In [9]:
cntr = 0
img_list = []
rel_list = []
attr_list = []
gqa_image_data = []
forb_list = open("non_ex_haha.txt", "r").read().split('\n')

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)
        
    img_list = create_obj_dict(img_list, obj_data, forb_list)
    cntr, rel_list = create_rel_dict(cntr, rel_list, obj_data, forb_list)
    gqa_image_data = create_gqa_image_data(gqa_image_data, obj_data, forb_list)
    attr_list = create_attr_dict(attr_list, obj_data, forb_list)

In [10]:
relsMap = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/relInfo.json'))
save_to_file(list(relsMap.keys()), 'gqa_relationship_list.txt')

In [11]:
attrInfo = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/attrMap.json'))
merged = list(itertools.chain(*list(attrInfo.values())))
save_to_file(merged, 'gqa_attribute_list.txt')

In [12]:
catMap = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/hObjInfo.json'))
save_to_file(list(catMap.keys()), 'gqa_object_list.txt')

In [13]:
with open('gqa_objects.json', 'w') as outfile:
    json.dump(img_list, outfile)
    
with open('gqa_relationships.json', 'w') as outfile:
    json.dump(rel_list, outfile)
    
with open('gqa_image_metadata.json', 'w') as outfile:  
    json.dump(gqa_image_data, outfile)
    
with open('gqa_attributes.json', 'w') as outfile:
    json.dump(attr_list, outfile)

In [38]:
obj_ent = []
att_ent = []
rel_ent = []

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)
    
    for key in tqdm(obj_data.keys()):
        for obj in obj_data[key]['objects']:
            
            if obj_data[key]['objects'][obj]['name'] in obj_ent:
                continue
            else: 
                obj_ent.append(obj_data[key]['objects'][obj]['name'])
            
            for rel in obj_data[key]['objects'][obj]['relations']:
                if rel['name'] == 'licking':
                    print('licking')
                if rel['name'] in rel_ent:
                    continue
                else: 
                    rel_ent.append(rel['name'])
                    
            for att in obj_data[key]['objects'][obj]['attributes']:
                if att in att_ent:
                    continue
                else: 
                    att_ent.append(att)

100%|██████████| 74942/74942 [00:03<00:00, 24782.53it/s]


In [40]:
len(rel_ent), len(att_ent)

(65, 182)